In [2]:
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd
import pandas_gbq as gbq
import requests
import time

project_id = 'arctic-task-238719'
private_key='arctic-task-238719-e6a1c5fe056b.json'
credentials = service_account.Credentials.from_service_account_file('./arctic-task-238719-e6a1c5fe056b.json')

gbq.context.credentials = credentials
gbq.context.project = project_id

In [125]:
# загрузить посты из определенной группе 
def upload_post_from_vk_group(username,group_id):
    token = 'e7a79876e7a79876e7a79876e9e7ce3561ee7a7e7a79876bb0457d3e507797f75821138'
    version = 5.92
    count = 100
    offset = 0
    all_posts = []
    x = 100
    df = pd.DataFrame(columns=['group_id','group_name', 'post_id','post','annotation','keywords'])
    response= requests.get('https://api.vk.com/method/groups.getById',
                            params={
                                'group_id': group_id,
                                'access_token': token,
                                'v': version,
                                
                            }
                            )

    group_name=response.json()['response'][0]['name']
    k=100
    while (k==100):
        start_time = time.time()
        if group_id.isnumeric():
            response = requests.get('https://api.vk.com/method/wall.get',
                                    params={
                                        'access_token': token,
                                        'v': version,
                                        'owner_id': '-'+group_id,
                                        'count': count,
                                        'offset':offset
                                    }
                                    )
        else:
            response = requests.get('https://api.vk.com/method/wall.get',
                                    params={
                                        'access_token': token,
                                        'v': version,
                                        'domain': group_id,
                                        'count': count,
                                        'offset':offset
                                    }
                                    )
        data = response.json()['response']['items']
        offset += 100
        k=len(data)
        j=0
        for i in range(k):
            annotation = 'annotation'
            keywords = 'keywords'
            post_id = data[i]['id']
            post = data[i]['text']
            if post!='':
                df.loc[j] = [group_id,group_name,int(post_id), post,annotation,keywords]
                j+=1
        gbq.to_gbq(df,'dataset.vk_storage_'+username, project_id , if_exists = 'append')
        time.sleep(0.1)
        


    
# upload_post_from_vk_group('kirill','192959150')  

In [46]:
# удалить из биб для vk группу по id 
def delete_row_group_from_userlibrary(username,group_id):
    credentials = service_account.Credentials.from_service_account_file('./arctic-task-238719-e6a1c5fe056b.json')

    client = bigquery.Client(project='arctic-task-238719',credentials=credentials)
    
    Query=f'DELETE  FROM dataset.vk_storage_{username} WHERE group_id = \'{group_id}\''
    query_job = client.query(Query)
    

delete_row_group_from_userlibrary('kirill','meaning_data')

In [99]:
# удалить из личной биб со статьями по названию и автору
def delete_row_article_from_userlibrary(username,author,title):
    credentials = service_account.Credentials.from_service_account_file('./arctic-task-238719-e6a1c5fe056b.json')

    client = bigquery.Client(project='arctic-task-238719',credentials=credentials)
    Query=f'DELETE  FROM dataset.{username} WHERE author like \'%{author}%\' and title likw \'%{title}%\' '
    query_job = client.query(Query)

# title='Рассеяние энергии механических колебаний в мягких ферромагнитных материалах в зависимости от фазы колебаний'
# delete_row_article_from_userlibrary('kirill','Глотова Людмила Сергеевна',title)

In [42]:
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import pymorphy2
from google.oauth2 import service_account
project_id = 'arctic-task-238719'
private_key='arctic-task-238719-e6a1c5fe056b.json'
import json
from google.cloud import bigquery
credentials = service_account.Credentials.from_service_account_file('./arctic-task-238719-e6a1c5fe056b.json')
from pandas.io import gbq
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))
import pandas as pd
morph=pymorphy2.MorphAnalyzer()
stemmer=SnowballStemmer('russian')
def search_in_user_vk_library(username,word='',mode='post'):
    
    if mode=='post_from_group':
        group_name,word=word.split(',')
        
        word = re.sub("[^а-яА-Яa-zA-Z0-9]", " ", word)
        words = word.lower().split()
        words = [w for w in words if not w in stops]
        words = [stemmer.stem(w) for w in words]
        if words=='':
            return "некорректный ввод"
        
        Query = f'SELECT * FROM dataset.vk_storage_{username}  WHERE group_name=\'{group_name}\' and (post LIKE \''
        for word in words:
            Query+='%{}'.format(word)
        Query +='%\' or post LIKE \' '
        flag=True
        for word in words:
            if flag==True:
                Query+='%{}'.format(word.capitalize())
                flag=False
            else:
                Query+='%{}'.format(word)
        Query +='%\')'
    
    
    
    
    word = re.sub("[^а-яА-Яa-zA-Z0-9]", " ", word)
    words = word.lower().split()
    words = [w for w in words if not w in stops]
    words = [stemmer.stem(w) for w in words]
    if words=='':
        return "некорректный ввод"

    if mode=='post':
        Query = f'SELECT * FROM dataset.vk_storage_{username}  WHERE post LIKE \''
        for word in words:
            Query+='%{}'.format(word)
        Query +='%\' or post LIKE \' '
        flag=True
        for word in words:
            if flag==True:
                Query+='%{}'.format(word.capitalize())
                flag=False
            else:
                Query+='%{}'.format(word)
        Query +='%\''
          
    df = gbq.read_gbq(Query, project_id, credentials=credentials)
    
    result = df.values.tolist()
    return result

# search(mode='kws',word='физика')



In [133]:

def show_all_groups(username):
    
    Query = f'SELECT  group_name,group_id FROM dataset.vk_storage_{username}  group by group_name,group_id'
    df = gbq.read_gbq(Query, project_id, credentials=credentials)
    result = df.values.tolist()
    
    return  result

# search(mode='kws',word='физика')

show_all_groups('kirill')

C:\python\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value for dialect is changing to "standard" in a future version. Pass in dialect="legacy" or set pandas_gbq.context.dialect="legacy" to disable this warning.
  """


[['hghg', '192959150'], ['Data Science', 'datascience']]

In [132]:

def show_user_library(username):
    
    Query = f'SELECT authors,title FROM dataset.{username}'
    df = gbq.read_gbq(Query, project_id, credentials=credentials)
    result = df.values.tolist()
    
    return  result

# search(mode='kws',word='физика')

# show_user_library(username='kirill')

In [6]:

def data_about_group_for_update(username,group_id,credentials=credentials):
    
    Query = f'SELECT group_id,MAX(post_id) as last FROM dataset.vk_storage_{username} where group_id=\'{group_id}\' group by group_id'
    df = gbq.read_gbq(Query, project_id, credentials=credentials)
    result = df['last'].values.tolist()
    
    return  result

# search(mode='kws',word='физика')

data_about_group_for_update(username='kirill',group_id='192959150')

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.42s/rows]


['5']

In [126]:
def update_post_from_vk_group(username,group_id,credentials):
    last_post_id = data_about_group_for_update(username,group_id,credentials)[0]
    
    token = 'e7a79876e7a79876e7a79876e9e7ce3561ee7a7e7a79876bb0457d3e507797f75821138'
    version = 5.92
    count = 100
    offset = 0
    all_posts = []
    x = 100
    df = pd.DataFrame(columns=['group_id','group_name', 'post_id','post','annotation','keywords'])
    response= requests.get('https://api.vk.com/method/groups.getById',
                            params={
                                'group_id': group_id,
                                'access_token': token,
                                'v': version,
                                
                            }
                            )

    group_name=response.json()['response'][0]['name']
    k=100
    while (k==100):
        start_time = time.time()
        if group_id.isnumeric():
            response = requests.get('https://api.vk.com/method/wall.get',
                                    params={
                                        'access_token': token,
                                        'v': version,
                                        'owner_id': '-'+group_id,
                                        'count': count,
                                        'offset':offset
                                    }
                                    )
        else:
            response = requests.get('https://api.vk.com/method/wall.get',
                                    params={
                                        'access_token': token,
                                        'v': version,
                                        'domain': group_id,
                                        'count': count,
                                        'offset':offset
                                    }
                                    )
        data = response.json()['response']['items']
        
        k=len(data)
        offset += 100
        j=0
        for i in range(len(data)):
            annotation = 'annotation'
            keywords = 'keywords'
            post_id = data[i]['id']
            post = data[i]['text']
            if post!='' and int(post_id)!=last_post_id:
                df.loc[j] = [group_id,group_name,int(post_id), post,annotation,keywords]
                j+=1
            elif int(post_id)==last_post_id:
                break
            
        
        if len(df)!=0:
            gbq.to_gbq(df,'dataset.vk_storage_'+username, project_id , if_exists = 'append')
        time.sleep(0.1)

# update_post_from_vk_group('kirill','192959150',credentials=credentials)

In [ ]:
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd
import pandas_gbq as gbq

project_id = 'arctic-task-238719'
private_key='arctic-task-238719-e6a1c5fe056b.json'
credentials = service_account.Credentials.from_service_account_file('./arctic-task-238719-e6a1c5fe056b.json')

gbq.context.credentials = credentials
gbq.context.project = project_id


def upload_user_bd(list_of_lists,username):
    try:
        df = pd.DataFrame(list_of_lists, columns=['author','title','keywords'])
        gbq.to_gbq(df,'dataset.'+username, project_id , if_exists = 'append'   )
    return True
    
    except:
        return -1